# Krungsri Auto Web Scraping

As far as AI chatbot is concerned, one of the main thing to do first and foremost is to prepare the data that would be used as a retriever for LLM model.
As we do not include any private internal data in this project, we have to use public data; which in this case, I decided to scrape information out of the Krungsri Auto website.
we would create a knowledge base about
1. Car for Cash in general : This support information can be scraped from ['www.car4cash.com'](www.car4cash.com) article section.
2. New car and Bigbike market price : This support information can be scrape from ['www.krungsrimarket.cjdataservice.com'](www.krungsrimarket.cjdataservice.com)

The workflow to scrape and prepare data as a retriever consists of
1. Set up Environment
2. Scrape Data from webpages
3. Embed Data (convert it to vector) and Store it in a Vectorstore (ChromaDB)


## 1. Set Up Environment 

we start by importing libraries, recalling OpenAI api key which will be used later for embedding data

In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import requests
import bs4
from tqdm import tqdm
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.document_loaders import CSVLoader


In [2]:
f = open(r".\openai_api_key.txt") # please provide your own openai api key if you want to use
user_api_key = f.read()

## 2. Scrape data

### 2.1 Scrape data from Krungsri Market website

In [3]:
def scrape_krungsrimkt(url) :

    response= get(url)
    soup=BeautifulSoup(response.text,'html.parser')

    product_brand = []
    product_price_tot = []
    product_price_month= []
    product_model = []
    product_cate = []

    for i in soup.find_all('div', class_='product-brand'):
        product_brand.append(i.get_text())
    for i in soup.find_all('div', class_='product-price-total'):
        product_price_tot.append(i.get_text())
    for i in soup.find_all('span', class_='value'):
        product_price_month.append(i.get_text())
    for i in soup.find_all('div', class_='product-model'):
        product_model.append(i.get_text())
    for i in soup.find_all('div', class_='product-category'):
        product_cate.append(i.get_text())

    product_cate = [i.replace('\n', '') for i in product_cate]

    product_price_tot = [i.strip('\n') for i in product_price_tot]
    product_price_tot = [i.replace(',', '') for i in product_price_tot]
    product_price_tot = [int(i) for i in product_price_tot]

    product_price_month = [i.replace(',', '') for i in product_price_month]
    product_price_month = [int(i) for i in product_price_month]   

    df = pd.DataFrame({'ยี่ห้อ': product_brand,'รุ่นรถ': product_model ,'กลุ่มรถ': product_cate,'ราคาเต็ม(บาท)':product_price_tot,'ราคาผ่อนต่อเดือนเริ่มต้น(บาท)': product_price_month})   

    return df


#### 2.1.1 Scraping Bigbike 

In [4]:
bigbike = []
for i in tqdm(range(1,58)):
    df = scrape_krungsrimkt(url = f'https://krungsrimarket.cjdataservice.com/bigbike/products?page={i}')
    bigbike.append(df)

100%|██████████| 57/57 [00:28<00:00,  1.97it/s]


In [5]:
bigbike_df = pd.concat(bigbike, axis = 0, ignore_index= True)
bigbike_df

,ยี่ห้อ,รุ่นรถ,กลุ่มรถ,ราคาเต็ม(บาท),ราคาผ่อนต่อเดือนเริ่มต้น(บาท)
0,Triumph,Speed 400 Cc My23,Speed,157900,2280
1,Triumph,Scrambler 400 Cc My23,Scrambler,179900,2598
2,Honda,Forza 350,Forza,173500,2505
3,Yamaha,Xmax 300 MY2022,Xmax,181000,2614
4,Honda,ADV 350 Urban Racer Edition,ADV,199500,2852
...,...,...,...,...,...
677,Bmw,F 900 XR Exclusive,F,550000,7473
678,Suzuki,V-Strom 1050 DE,V-Strom,619000,8486
679,Bmw,R 1250 gs adventure สีดำ triple black,R,1150000,15624
680,Moto Guzzi,V7 special,V7,649000,8818


#### 2.1.2 Scraping New Car

In [6]:
newcar = []
for i in tqdm(range(1,71)):
    df = scrape_krungsrimkt(url = f'https://krungsrimarket.cjdataservice.com/newcar/products?kw=&page={i}')
    newcar.append(df)

100%|██████████| 70/70 [00:34<00:00,  2.03it/s]


In [7]:
newcar_df = pd.concat(newcar, axis = 0, ignore_index= True)
newcar_df

,ยี่ห้อ,รุ่นรถ,กลุ่มรถ,ราคาเต็ม(บาท),ราคาผ่อนต่อเดือนเริ่มต้น(บาท)
0,MITSUBISHI,ATTRAGE 1.2 Active CVT AT,Passenger,529000,5848
1,MITSUBISHI,ATTRAGE 1.2 GLX CVT AT Special Edition,Passenger,543000,6003
2,MITSUBISHI,ATTRAGE 1.2 SMART CVT,Passenger,584000,6456
3,MITSUBISHI,TRITON D-Cab 2.4 Ultra AT 4WD,Pickup 4 drs.,1228000,14181
4,MITSUBISHI,TRITON D-Cab Plus 2.4 Prime,Pickup 4 drs.,893000,10312
...,...,...,...,...,...
824,VOLVO,XC60 XC60 T8 Twin Engine AWD Momentum,Passenger,3290000,36367
825,VOLVO,XC60 XC60 T8 Twin Engine AWD R-Desingn,Passenger,3590000,39683
826,VOLVO,XC60 XC60 T8 Twin Engine AWD R-Desingn,Passenger,3590000,39683
827,BENZ,SL SL 400 AMG Premium,Passenger,16720000,184816


#### 2.1.3 Concatenate Bigbike and New Car DF together

In [8]:
bigbike_df['ชนิดรถ'] = 'บิ๊กไบค์ (ฺBigBike)'
newcar_df['ชนิดรถ'] = 'รถยนต์ (ฺNew Car)'

In [9]:
df = pd.concat([bigbike_df[['ชนิดรถ','ยี่ห้อ','รุ่นรถ','กลุ่มรถ','ราคาเต็ม(บาท)','ราคาผ่อนต่อเดือนเริ่มต้น(บาท)']], newcar_df[['ชนิดรถ','ยี่ห้อ','รุ่นรถ','กลุ่มรถ','ราคาเต็ม(บาท)','ราคาผ่อนต่อเดือนเริ่มต้น(บาท)']]], axis = 0)
df.reset_index(drop = True, inplace = True)
df

,ชนิดรถ,ยี่ห้อ,รุ่นรถ,กลุ่มรถ,ราคาเต็ม(บาท),ราคาผ่อนต่อเดือนเริ่มต้น(บาท)
0,บิ๊กไบค์ (ฺBigBike),Triumph,Speed 400 Cc My23,Speed,157900,2280
1,บิ๊กไบค์ (ฺBigBike),Triumph,Scrambler 400 Cc My23,Scrambler,179900,2598
2,บิ๊กไบค์ (ฺBigBike),Honda,Forza 350,Forza,173500,2505
3,บิ๊กไบค์ (ฺBigBike),Yamaha,Xmax 300 MY2022,Xmax,181000,2614
4,บิ๊กไบค์ (ฺBigBike),Honda,ADV 350 Urban Racer Edition,ADV,199500,2852
...,...,...,...,...,...,...
1506,รถยนต์ (ฺNew Car),VOLVO,XC60 XC60 T8 Twin Engine AWD Momentum,Passenger,3290000,36367
1507,รถยนต์ (ฺNew Car),VOLVO,XC60 XC60 T8 Twin Engine AWD R-Desingn,Passenger,3590000,39683
1508,รถยนต์ (ฺNew Car),VOLVO,XC60 XC60 T8 Twin Engine AWD R-Desingn,Passenger,3590000,39683
1509,รถยนต์ (ฺNew Car),BENZ,SL SL 400 AMG Premium,Passenger,16720000,184816


In [10]:
#save dataset (as backup)
df.to_csv('./DataSource/car_price_info.csv',index = False)

In [11]:
#convert to CSV to Langchain document object
loader = CSVLoader('./DataSource/car_price_info.csv', encoding= 'utf-8')
car_price_info_docs = loader.load()

### 2.2 Scrape Article about Car4Cash

In [12]:
def scrape_web_article(url, tag, class_ ):
    response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(response.text,features="html.parser")

    contents = []
    for i in soup.find_all(tag,class_ = class_):
        contents.append(i.get_text())
    
    full_content = '\n'.join(contents)
    
    return full_content

In [13]:
#list of all C4C articles on the web
c4c_articles = ['https://www.car4cash.com/contents/financial/what-is-auto-loan-refinancing',
 'https://www.car4cash.com/contents/financial/go-digital-lending',
'https://www.car4cash.com/contents/financial/go-by-krungsri-auto-application',
'https://www.car4cash.com/contents/financial/register-a-company',
'https://www.car4cash.com/contents/financial/differentiation-strategy',
'https://www.car4cash.com/contents/financial/emergency-fund',
'https://www.car4cash.com/contents/financial/2-things-you-should-know-before-refinance',
'https://www.car4cash.com/contents/financial/techniques-to-manage-debt-quickly',
'https://www.car4cash.com/contents/financial/types-of-auto-loans',
'https://www.car4cash.com/contents/financial/refinance-car-how-to-most-efficiently',
'https://www.car4cash.com/contents/financial/5-signs-you-re-ready-expand-your-business',
'https://www.car4cash.com/contents/financial/8-ways-to-prepare-upsell',
'https://www.car4cash.com/contents/financial/5-question-for-refinance-motorcycle',
'https://www.car4cash.com/contents/financial/how-to-refinance-with-car4cash',
'https://www.car4cash.com/contents/financial/5-technique-to-save-on-car-insurance',
'https://www.car4cash.com/contents/product/car-loan-refinancing',
'https://www.car4cash.com/contents/product/tax-and-compulsory-motor-insurance-renewal-process',
'https://www.car4cash.com/contents/financial/the-best-app-for-money-planning',
'https://www.car4cash.com/contents/product/different-types-of-car-refinances',
'https://www.car4cash.com/contents/finance/filling-claim-impact-car-insurance-premium',
'https://www.car4cash.com/contents/financial/make-extra-money-with-your-car',
'https://www.car4cash.com/contents/product/inspect-used-car-before-buying',
'https://www.car4cash.com/contents/product/car-loan-for-pay-off-debt',
'https://www.car4cash.com/contents/product/car-refinance-boost-liquidity',
'https://www.car4cash.com/contents/product/car-refinance-during-covid-19',
'https://www.car4cash.com/contents/product/car-refinance-for-express-cash']

In [14]:
contents_c4c = []
for i in tqdm(c4c_articles):
    content = scrape_web_article(i,tag = 'div',class_ = 'content-detail')
    contents_c4c.append(content)

100%|██████████| 26/26 [00:05<00:00,  4.72it/s]


In [15]:
contents_c4c

['รีไฟแนนซ์รถคืออะไร เลือกรีไฟแนนซ์รถที่ไหนดี 19 กันยายน 2566  ในปัจจุบันที่เรากำลังเผชิญกับผลกระทบทางเศรษฐกิจที่ไม่สามารถหลีกเลี่ยงได้ เรื่องการเงินกลายเป็นปัญหาที่เกิดขึ้นกับทุกคน ส่งผลให้การเงินของเพื่อนๆ อาจขาดสภาพคล่อง การรีไฟแนนซ์รถเป็นหนึ่งทางเลือกที่ดี เพื่อจัดการกับปัญหาการเงินที่เกิดขึ้น รวมถึงช่วยลดภาระค่าใช้จ่ายของเพื่อนๆ ได้ วันนี้เราจะมาทำความรู้จักกันว่ารีไฟแนนซ์รถคืออะไร พร้อมบอกเคล็ดลับการเลือกรีไฟแนนซ์รถที่ไหนดีให้เพื่อนๆ ทราบกัน          รีไฟแนนซ์รถคืออะไร          การรีไฟแนนซ์รถ คือการนำรถมาเป็นหลักทรัพย์ค้ำประกันเพื่อขอสินเชื่อ สำหรับผู้ที่ต้องการเงินฉุกเฉิน หรือต้องการเงินก้อนเพื่อนำไปต่อยอดทางธุรกิจ สามารถเลือกได้ว่า ต้องการดอกเบี้ยคงที่ หรือดอกเบี้ยลดต้นลดดอก เพื่อช่วยให้สภาพทางการเงินยังคล่องตัวต่อไปได้ โดยที่ยังมีรถคู่ใจใช้งานอยู่เหมือนเดิม          ข้อดีของการรีไฟแนนซ์รถคืออะไร          หากเพื่อนๆ รู้สึกว่าสถานภาพทางการเงินเริ่มตึงมือ อาจถึงเวลาที่จะใช้การรีไฟแนนซ์รถเข้ามาช่วยกอบกู้สถานภาพทางการเงิน เพราะข้อดีของการรีไฟแนนซ์นั้นสามารถช่วยลดค่างวดต่อเดือนลง ช่

In [16]:
#split contents into chunks of texts
def split_text(string,chunk_size = 1000, chunk_overlap = 0):

    character_splitter = RecursiveCharacterTextSplitter(
        separators= ["\n\n", "\n", " ",""],
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap
        )
    
    texts = character_splitter.split_text(string)
    print(f'total chunks: {len(texts)}')
    
    return texts

In [17]:
#convert string to langchain document object
def convert_to_docs(texts):
    docs = []
    for text in tqdm( texts ):
        doc = Document(page_content= text, metadata={'source' : 'local'})
        docs.append(doc)
    return docs

In [18]:
texts_c4c = []
for i in contents_c4c:
    text = split_text(i)
    texts_c4c += text

total chunks: 8
total chunks: 4
total chunks: 5
total chunks: 5
total chunks: 4
total chunks: 4
total chunks: 3
total chunks: 6
total chunks: 7
total chunks: 4
total chunks: 6
total chunks: 6
total chunks: 6
total chunks: 4
total chunks: 5
total chunks: 3
total chunks: 9
total chunks: 6
total chunks: 4
total chunks: 3
total chunks: 4
total chunks: 4
total chunks: 3
total chunks: 5
total chunks: 3
total chunks: 4


In [19]:
texts_c4c

['รีไฟแนนซ์รถคืออะไร เลือกรีไฟแนนซ์รถที่ไหนดี 19 กันยายน 2566  ในปัจจุบันที่เรากำลังเผชิญกับผลกระทบทางเศรษฐกิจที่ไม่สามารถหลีกเลี่ยงได้ เรื่องการเงินกลายเป็นปัญหาที่เกิดขึ้นกับทุกคน ส่งผลให้การเงินของเพื่อนๆ อาจขาดสภาพคล่อง การรีไฟแนนซ์รถเป็นหนึ่งทางเลือกที่ดี เพื่อจัดการกับปัญหาการเงินที่เกิดขึ้น รวมถึงช่วยลดภาระค่าใช้จ่ายของเพื่อนๆ ได้ วันนี้เราจะมาทำความรู้จักกันว่ารีไฟแนนซ์รถคืออะไร พร้อมบอกเคล็ดลับการเลือกรีไฟแนนซ์รถที่ไหนดีให้เพื่อนๆ ทราบกัน          รีไฟแนนซ์รถคืออะไร          การรีไฟแนนซ์รถ คือการนำรถมาเป็นหลักทรัพย์ค้ำประกันเพื่อขอสินเชื่อ สำหรับผู้ที่ต้องการเงินฉุกเฉิน หรือต้องการเงินก้อนเพื่อนำไปต่อยอดทางธุรกิจ สามารถเลือกได้ว่า ต้องการดอกเบี้ยคงที่ หรือดอกเบี้ยลดต้นลดดอก เพื่อช่วยให้สภาพทางการเงินยังคล่องตัวต่อไปได้ โดยที่ยังมีรถคู่ใจใช้งานอยู่เหมือนเดิม          ข้อดีของการรีไฟแนนซ์รถคืออะไร          หากเพื่อนๆ รู้สึกว่าสถานภาพทางการเงินเริ่มตึงมือ อาจถึงเวลาที่จะใช้การรีไฟแนนซ์รถเข้ามาช่วยกอบกู้สถานภาพทางการเงิน เพราะข้อดีของการรีไฟแนนซ์นั้นสามารถช่วยลดค่างวดต่อเดือนลง',


In [20]:
docs_c4c = convert_to_docs(texts_c4c)

100%|██████████| 125/125 [00:00<00:00, 41311.80it/s]


In [21]:
docs_c4c

[Document(page_content='รีไฟแนนซ์รถคืออะไร เลือกรีไฟแนนซ์รถที่ไหนดี 19 กันยายน 2566  ในปัจจุบันที่เรากำลังเผชิญกับผลกระทบทางเศรษฐกิจที่ไม่สามารถหลีกเลี่ยงได้ เรื่องการเงินกลายเป็นปัญหาที่เกิดขึ้นกับทุกคน ส่งผลให้การเงินของเพื่อนๆ อาจขาดสภาพคล่อง การรีไฟแนนซ์รถเป็นหนึ่งทางเลือกที่ดี เพื่อจัดการกับปัญหาการเงินที่เกิดขึ้น รวมถึงช่วยลดภาระค่าใช้จ่ายของเพื่อนๆ ได้ วันนี้เราจะมาทำความรู้จักกันว่ารีไฟแนนซ์รถคืออะไร พร้อมบอกเคล็ดลับการเลือกรีไฟแนนซ์รถที่ไหนดีให้เพื่อนๆ ทราบกัน          รีไฟแนนซ์รถคืออะไร          การรีไฟแนนซ์รถ คือการนำรถมาเป็นหลักทรัพย์ค้ำประกันเพื่อขอสินเชื่อ สำหรับผู้ที่ต้องการเงินฉุกเฉิน หรือต้องการเงินก้อนเพื่อนำไปต่อยอดทางธุรกิจ สามารถเลือกได้ว่า ต้องการดอกเบี้ยคงที่ หรือดอกเบี้ยลดต้นลดดอก เพื่อช่วยให้สภาพทางการเงินยังคล่องตัวต่อไปได้ โดยที่ยังมีรถคู่ใจใช้งานอยู่เหมือนเดิม          ข้อดีของการรีไฟแนนซ์รถคืออะไร          หากเพื่อนๆ รู้สึกว่าสถานภาพทางการเงินเริ่มตึงมือ อาจถึงเวลาที่จะใช้การรีไฟแนนซ์รถเข้ามาช่วยกอบกู้สถานภาพทางการเงิน เพราะข้อดีของการรีไฟแนนซ์นั้นสามารถช่ว

### 2.3 Scrape Krungsri Auto AboutUS webpage

We want to LLM to know basic information about KrungsriAuto

In [22]:
#list of all About us on the KrungsriAuto web
aboutus_articles = ['https://www.krungsriauto.com/auto/About-Us/Krungsri-Auto.html',
'https://www.krungsriauto.com/auto/About-Us/typeof.html',
'https://www.krungsriauto.com/auto/About-Us/Service-Channel.html',
'https://www.krungsriauto.com/auto/About-Us/Awards.html',
'https://www.krungsriauto.com/auto/About-Us/Responsible-credit-policy.html',
'https://www.krungsriauto.com/auto/About-Us/Privacy_Notice.html']

In [23]:
contents_aboutus = []
for i in tqdm(aboutus_articles):
    content = scrape_web_article(i,tag = 'div',class_ = 'about-details')
    contents_aboutus.append(content)

100%|██████████| 6/6 [00:01<00:00,  3.96it/s]


In [24]:
contents_aboutus

['\n\nกรุงศรี ออโต้\nเกี่ยวกับบริษัท\nประสบการณ์ใหม่กับ กรุงศรี ออโต้\n\xa0 \xa0 \xa0 \xa0 “กรุงศรี ออโต้” ผู้นำธุรกิจสินเชื่อยานยนต์ เครือธนาคารกรุงศรีอยุธยา จำกัด (มหาชน) ให้บริการสินเชื่อยานยนต์ครบวงจร ได้แก่ สินเชื่อเพื่อคนมีรถ “คาร์ ฟอร์ แคช”\xa0 และ “คาร์ ฟอร์ แคช โปะ”\xa0 \xa0สินเชื่อรถบ้าน ”กรุงศรี รถบ้าน” สินเชื่อรถใหม่ “กรุงศรี นิว คาร์” สินเชื่อรถเต็นท์ “กรุงศรี ยูสด์ คาร์” สินเชื่อรถบรรทุกใหม่ “กรุงศรี ทรัค” ซึ่งให้บริการโดยกลุ่มงานธุรกิจสินเชื่อยานยนต์ ธนาคารกรุงศรีอยุธยา จำกัด (มหาชน) สินเชื่อรถจักรยานยนต์ “กรุงศรี มอเตอร์ไซค์” สินเชื่อบิ๊ก ไบค์\xa0 “กรุงศรี บิ๊ก ไบค์” สินเชื่อบิ๊ก ไบค์ มือสอง “กรุงศรี ยูสด์ บิ๊ก ไบค์” สินเชื่อเพื่อคนมีรถ “คาร์ ฟอร์ แคช มอเตอร์ไซค์” สินเชื่อผู้แทนจำหน่ายรถยนต์ “กรุงศรี อินเวนทอรี่ ไฟแนนซ์” รวมถึงบริการด้านการประกันภัย “กรุงศรี ออโต้ โบรคเกอร์” ซึ่งให้บริการโดยบริษัท อยุธยา แคปปิตอล ออโต้ ลีส จำกัด (มหาชน)\n\r\nลูกค้าสามารถเลือกใช้บริการสินเชื่อของ กรุงศรี ออโต้ พร้อมรับคำปรึกษาทั้งเรื่องรถและเรื่องเงิน ช่วยให้เรื่องเงินเป็นเรื่องง่าย ผ่าน

In [25]:
texts_aboutus = []
for i in contents_aboutus:
    text = split_text(i)
    texts_aboutus += text

total chunks: 6
total chunks: 3
total chunks: 3
total chunks: 39
total chunks: 1
total chunks: 9


In [26]:
texts_aboutus

['กรุงศรี ออโต้\nเกี่ยวกับบริษัท\nประสบการณ์ใหม่กับ กรุงศรี ออโต้\n\xa0 \xa0 \xa0 \xa0 “กรุงศรี ออโต้” ผู้นำธุรกิจสินเชื่อยานยนต์ เครือธนาคารกรุงศรีอยุธยา จำกัด (มหาชน) ให้บริการสินเชื่อยานยนต์ครบวงจร ได้แก่ สินเชื่อเพื่อคนมีรถ “คาร์ ฟอร์ แคช”\xa0 และ “คาร์ ฟอร์ แคช โปะ”\xa0 \xa0สินเชื่อรถบ้าน ”กรุงศรี รถบ้าน” สินเชื่อรถใหม่ “กรุงศรี นิว คาร์” สินเชื่อรถเต็นท์ “กรุงศรี ยูสด์ คาร์” สินเชื่อรถบรรทุกใหม่ “กรุงศรี ทรัค” ซึ่งให้บริการโดยกลุ่มงานธุรกิจสินเชื่อยานยนต์ ธนาคารกรุงศรีอยุธยา จำกัด (มหาชน) สินเชื่อรถจักรยานยนต์ “กรุงศรี มอเตอร์ไซค์” สินเชื่อบิ๊ก ไบค์\xa0 “กรุงศรี บิ๊ก ไบค์” สินเชื่อบิ๊ก ไบค์ มือสอง “กรุงศรี ยูสด์ บิ๊ก ไบค์” สินเชื่อเพื่อคนมีรถ “คาร์ ฟอร์ แคช มอเตอร์ไซค์” สินเชื่อผู้แทนจำหน่ายรถยนต์ “กรุงศรี อินเวนทอรี่ ไฟแนนซ์” รวมถึงบริการด้านการประกันภัย “กรุงศรี ออโต้ โบรคเกอร์” ซึ่งให้บริการโดยบริษัท อยุธยา แคปปิตอล ออโต้ ลีส จำกัด (มหาชน)',
 'ลูกค้าสามารถเลือกใช้บริการสินเชื่อของ กรุงศรี ออโต้ พร้อมรับคำปรึกษาทั้งเรื่องรถและเรื่องเงิน ช่วยให้เรื่องเงินเป็นเรื่องง่าย ผ่านสาขา 

In [27]:
docs_aboutus = convert_to_docs(texts_aboutus)

100%|██████████| 61/61 [00:00<00:00, 75495.00it/s]


In [28]:
docs_aboutus

[Document(page_content='กรุงศรี ออโต้\nเกี่ยวกับบริษัท\nประสบการณ์ใหม่กับ กรุงศรี ออโต้\n\xa0 \xa0 \xa0 \xa0 “กรุงศรี ออโต้” ผู้นำธุรกิจสินเชื่อยานยนต์ เครือธนาคารกรุงศรีอยุธยา จำกัด (มหาชน) ให้บริการสินเชื่อยานยนต์ครบวงจร ได้แก่ สินเชื่อเพื่อคนมีรถ “คาร์ ฟอร์ แคช”\xa0 และ “คาร์ ฟอร์ แคช โปะ”\xa0 \xa0สินเชื่อรถบ้าน ”กรุงศรี รถบ้าน” สินเชื่อรถใหม่ “กรุงศรี นิว คาร์” สินเชื่อรถเต็นท์ “กรุงศรี ยูสด์ คาร์” สินเชื่อรถบรรทุกใหม่ “กรุงศรี ทรัค” ซึ่งให้บริการโดยกลุ่มงานธุรกิจสินเชื่อยานยนต์ ธนาคารกรุงศรีอยุธยา จำกัด (มหาชน) สินเชื่อรถจักรยานยนต์ “กรุงศรี มอเตอร์ไซค์” สินเชื่อบิ๊ก ไบค์\xa0 “กรุงศรี บิ๊ก ไบค์” สินเชื่อบิ๊ก ไบค์ มือสอง “กรุงศรี ยูสด์ บิ๊ก ไบค์” สินเชื่อเพื่อคนมีรถ “คาร์ ฟอร์ แคช มอเตอร์ไซค์” สินเชื่อผู้แทนจำหน่ายรถยนต์ “กรุงศรี อินเวนทอรี่ ไฟแนนซ์” รวมถึงบริการด้านการประกันภัย “กรุงศรี ออโต้ โบรคเกอร์” ซึ่งให้บริการโดยบริษัท อยุธยา แคปปิตอล ออโต้ ลีส จำกัด (มหาชน)', metadata={'source': 'local'}),
 Document(page_content='ลูกค้าสามารถเลือกใช้บริการสินเชื่อของ กรุงศรี ออโต้ พร้อมรับ

## 3. Embed All scraped documents and Store it in Vectorstore


In [29]:
docs = car_price_info_docs + docs_c4c + docs_aboutus

In [30]:
len(docs)

1697

In [31]:
embedding_function = OpenAIEmbeddings(api_key= user_api_key)

c:\Users\11432382\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


db is the database that collect all documents together

In [32]:
db = Chroma.from_documents(docs, embedding_function, persist_directory=r'./DataSource/ka_web_base_knowlege_from_scraping.db')

In [33]:
db.persist()

car_price_info_db is the database that collect only documents about car/big bike price info.

In [34]:
car_price_info_db = Chroma.from_documents(car_price_info_docs, embedding_function, persist_directory=r'./DataSource/car_price_knowlege_from_scraping.db')
car_price_info_db.persist()

article_info_db is the database that collect only documents about Car4Cash article and basic information about KrungsriAuto.

In [35]:
article_info_db = Chroma.from_documents(docs_c4c + docs_aboutus, embedding_function, persist_directory=r'./DataSource/article_info_knowlege_from_scraping.db')
article_info_db.persist()